# 50 - Experiments 1 and 2 : figures and tables

In [1]:
import os, sys
from pathlib import Path

ENV_IS_GOOGLE_COLAB = True if 'google.colab' in str(get_ipython()) else False
os.environ["ENV_IS_GOOGLE_COLAB"] = str(ENV_IS_GOOGLE_COLAB)

if ENV_IS_GOOGLE_COLAB:
  from google.colab import drive
  mountpoint = Path("/content/drive")
  drive.mount(str(mountpoint)) # Mount gdrive to BASE
  base = mountpoint / "MyDrive/article_icdar_2023" # Adapt this to your situation
  sys.path.append(str(base)) # Add BASE to Python Path
  BASE = Path(base).resolve() # Make BASE absolute
  DATASETS =  BASE / "dataset"
else:
  BASE = Path(os.path.dirname(os.path.realpath("__file__"))).resolve() # If not on GColab, BASE will be the directory of this notebook
  DATASETS = Path('/work/stual/dataset_ICDAR')
  OUT_BASE = Path('/work/stual/res_ICDAR/method_0')

print(sys.path)
print(BASE)
print(DATASETS)
print(OUT_BASE)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
['/lrde/home2/stual/stage_DAS/m0_flat_ner', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages']
/lrde/home2/stual/stage_DAS/m0_flat_ner
/work/stual/dataset_ICDAR
/work/stual/res_ICDAR/method_0


In [2]:
MAX_TRAINSET_SIZE = 6084

## Tools

In [3]:
import pandas as pd
import re
import os
import json

def compile_metrics(path): 
    rundirs = [f for f in os.listdir(path)]

    df = pd.DataFrame()
    for run_dir in rundirs:
        if 'run' in run_dir:
            run_path = path / run_dir
            nrun = re.search("\d+",run_dir)[0]

            files = [f for f in os.listdir(run_path) if "test_" in f and 'json' in f]
            sizes = [int(re.search("\d+",f)[0]) for f in files]

            for file, size in zip(files,sizes):
                file_path = run_path / file
                dftmp = pd.read_json(file_path, typ='series')
                dftmp = pd.DataFrame([dftmp])

                dftmp["trainsize"] = size 
                dftmp["run"] = nrun
                dftmp["trainsize_p"] = round(100 * size / MAX_TRAINSET_SIZE, 1)
                df = pd.concat([df, dftmp])

    return df.groupby(["run","trainsize"]).first()

## Path

In [4]:
from pathlib import Path

# PATHS
METRICS_DIR_E1 = OUT_BASE / "20-experiment_1_metrics"
METRICS_DIR_E2 = OUT_BASE / "40-experiment_2_metrics"

# 31 - Global metrics

In [6]:
res_ref = []
keys = []

if os.path.exists(METRICS_DIR_E1 / "21-flat-ner-ref-camembert_ner/run_2"):
    camembert = compile_metrics(METRICS_DIR_E1 / "21-flat-ner-ref-camembert_ner")
    res_ref.append(camembert)
    keys.append("CmBERT")

if os.path.exists(METRICS_DIR_E1 / "22-flat-ner-ref-pretrained_camembert_ner/run_2"):
    ptrn_camembert = compile_metrics(METRICS_DIR_E1 / "22-flat-ner-ref-pretrained_camembert_ner")
    res_ref.append(ptrn_camembert)
    keys.append("CmBERT+ptrn")
    
metrics_raw_ref = pd.concat(res_ref, keys=keys,names=["Test"])
metrics_raw_ref

eval_loss  eval_precision  eval_recall   eval_f1  \
Test        run trainsize                                                     
CmBERT      1   6084        0.147203        0.964559     0.964107  0.964333   
            2   6084        0.167061        0.966646     0.972378  0.969504   
            3   6084        0.165029        0.966822     0.973159  0.969980   
            4   6084        0.150362        0.966521     0.968633  0.967576   
            5   6084        0.181062        0.969370     0.968009  0.968689   
CmBERT+ptrn 1   6084        0.128630        0.968658     0.974251  0.971446   
            2   6084        0.134774        0.964684     0.971910  0.968284   
            3   6084        0.114306        0.963670     0.968633  0.966145   
            4   6084        0.124356        0.960155     0.966448  0.963291   
            5   6084        0.125162        0.966466     0.975968  0.971193   

                           eval_accuracy  \
Test        run trainsize                  
CmBERT      1   6084            0.956089   
            2   6084            0.956244   
            3   6084            0.958050   
            4   6084            0.954334   
            5   6084            0.955212   
CmBERT+ptrn 1   6084            0.957276   
            2   6084            0.956192   
            3   6084            0.956863   
            4   6084            0.952477   
            5   6084            0.956656   

                                                                    eval_PER  \
Test        run trainsize                                                      
CmBERT      1   6084       {'precision': 0.965761511216056, 'recall': 0.9...   
            2   6084       {'precision': 0.9711255156157921, 'recall': 0....   
            3   6084       {'precision': 0.9793144208037821, 'recall': 0....   
            4   6084       {'precision': 0.9711084905660371, 'recall': 0....   
            5   6084       {'precision': 0.9709715639810421, 'recall': 0....   
CmBERT+ptrn 1   6084       {'precision': 0.9775811209439521, 'recall': 0....   
            2   6084       {'precision': 0.9741176470588231, 'recall': 0....   
            3   6084       {'precision': 0.9700352526439481, 'recall': 0....   
            4   6084       {'precision': 0.9595545134818281, 'recall': 0....   
            5   6084       {'precision': 0.9864146485528641, 'recall': 0....   

                                                                    eval_ACT  \
Test        run trainsize                                                      
CmBERT      1   6084       {'precision': 0.9128440366972471, 'recall': 0....   
            2   6084       {'precision': 0.9204339963833631, 'recall': 0....   
            3   6084       {'precision': 0.9077887197851381, 'recall': 0....   
            4   6084       {'precision': 0.9202566452795601, 'recall': 0....   
            5   6084       {'precision': 0.928305400372439, 'recall': 0.9...   
CmBERT+ptrn 1   6084       {'precision': 0.9207207207207201, 'recall': 0....   
            2   6084       {'precision': 0.910152740341419, 'recall': 0.9...   
            3   6084       {'precision': 0.9072907290729071, 'recall': 0....   
            4   6084       {'precision': 0.9005424954792041, 'recall': 0....   
            5   6084       {'precision': 0.906607929515418, 'recall': 0.9...   

                                                                    eval_LOC  \
Test        run trainsize                                                      
CmBERT      1   6084       {'precision': 0.97265625, 'recall': 0.97537772...   
            2   6084       {'precision': 0.9693763919821821, 'recall': 0....   
            3   6084       {'precision': 0.9682981090100111, 'recall': 0....   
            4   6084       {'precision': 0.9688369504730101, 'recall': 0....   
            5   6084       {'precision': 0.9710950528071151, 'recall': 0....   
CmBERT+ptrn 1   6084       {'precision': 0.9705228031145711, 'recal

In [7]:
res_ocr = []
keys = []
    
if os.path.exists(METRICS_DIR_E2 / "41-flat-ner-pero-ocr-camembert_ner/run_2"):
    camembert_p = compile_metrics(METRICS_DIR_E2 / "41-flat-ner-pero-ocr-camembert_ner")
    res_ocr.append(camembert_p)
    keys.append("CmBERT")

if os.path.exists(METRICS_DIR_E2 / "42-flat-ner-pero-ocr-pretrained_camembert_ner/run_2"):
    ptrn_camembert_p = compile_metrics(METRICS_DIR_E2 / "42-flat-ner-pero-ocr-pretrained_camembert_ner")
    res_ocr.append(ptrn_camembert_p)
    keys.append("CmBERT+ptrn")
    
metrics_raw_ocr = pd.concat(res_ocr, keys=keys,names=["Test"])
metrics_raw_ocr

eval_loss  eval_precision  eval_recall   eval_f1  \
Test        run trainsize                                                     
CmBERT      1   6084        0.215006        0.944202     0.945528  0.944865   
            2   6084        0.248555        0.945568     0.948962  0.947262   
            3   6084        0.248467        0.947140     0.953644  0.950381   
            4   6084        0.210600        0.946724     0.954113  0.950404   
            5   6084        0.199033        0.941918     0.944124  0.943020   
CmBERT+ptrn 1   6084        0.206024        0.942260     0.957703  0.949919   
            2   6084        0.158690        0.946911     0.952084  0.949490   
            3   6084        0.174147        0.947524     0.949742  0.948632   
            4   6084        0.190652        0.943963     0.951771  0.947851   
            5   6084        0.170805        0.943533     0.951928  0.947712   

                           eval_accuracy  \
Test        run trainsize                  
CmBERT      1   6084            0.937099   
            2   6084            0.945868   
            3   6084            0.945451   
            4   6084            0.945973   
            5   6084            0.937099   
CmBERT+ptrn 1   6084            0.944981   
            2   6084            0.945294   
            3   6084            0.945607   
            4   6084            0.942058   
            5   6084            0.947956   

                                                                    eval_PER  \
Test        run trainsize                                                      
CmBERT      1   6084       {'precision': 0.9445427728613561, 'recall': 0....   
            2   6084       {'precision': 0.9555687203791471, 'recall': 0....   
            3   6084       {'precision': 0.9540094339622641, 'recall': 0....   
            4   6084       {'precision': 0.9517363154796941, 'recall': 0....   
            5   6084       {'precision': 0.9468399291199051, 'recall': 0....   
CmBERT+ptrn 1   6084       {'precision': 0.959435626102292, 'recall': 0.9...   
            2   6084       {'precision': 0.9605882352941171, 'recall': 0....   
            3   6084       {'precision': 0.9650680876258141, 'recall': 0....   
            4   6084       {'precision': 0.9479227618490341, 'recall': 0....   
            5   6084       {'precision': 0.9512338425381901, 'recall': 0....   

                                                                    eval_ACT  \
Test        run trainsize                                                      
CmBERT      1   6084       {'precision': 0.8964577656675751, 'recall': 0....   
            2   6084       {'precision': 0.887892376681614, 'recall': 0.9...   
            3   6084       {'precision': 0.892511013215859, 'recall': 0.9...   
            4   6084       {'precision': 0.9003558718861211, 'recall': 0....   
            5   6084       {'precision': 0.898458748866727, 'recall': 0.9...   
CmBERT+ptrn 1   6084       {'precision': 0.8733850129198961, 'recall': 0....   
            2   6084       {'precision': 0.8877005347593581, 'recall': 0....   
            3   6084       {'precision': 0.87677304964539, 'recall': 0.90...   
            4   6084       {'precision': 0.888790035587188, 'recall': 0.9...   
            5   6084       {'precision': 0.8793256433007981, 'recall': 0....   

                                                                    eval_LOC  \
Test        run trainsize                                                      
CmBERT      1   6084       {'precision': 0.9392265193370161, 'recall': 0....   
            2   6084       {'precision': 0.942745969983324, 'recall': 0.9...   
            3   6084       {'precision': 0.9415656008820281, 'recall': 0....   
            4   6084       {'precision': 0.9425731639977911, 'recall': 0....   
            5   6084       {'precision': 0.940720221606648, 'recall': 0.9...   
CmBERT+ptrn 1   6084       {'precision': 0.9398786541643681, 'recal

In [8]:
datasets = [metrics_raw_ref,metrics_raw_ocr]
keys = ["Reference","OCR"]
metrics_raw = pd.concat(datasets, keys=keys,names=["Dataset"])
metrics_raw

eval_loss  eval_precision  eval_recall  \
Dataset   Test        run trainsize                                           
Reference CmBERT      1   6084        0.147203        0.964559     0.964107   
                      2   6084        0.167061        0.966646     0.972378   
                      3   6084        0.165029        0.966822     0.973159   
                      4   6084        0.150362        0.966521     0.968633   
                      5   6084        0.181062        0.969370     0.968009   
          CmBERT+ptrn 1   6084        0.128630        0.968658     0.974251   
                      2   6084        0.134774        0.964684     0.971910   
                      3   6084        0.114306        0.963670     0.968633   
                      4   6084        0.124356        0.960155     0.966448   
                      5   6084        0.125162        0.966466     0.975968   
OCR       CmBERT      1   6084        0.215006        0.944202     0.945528   
                      2   6084        0.248555        0.945568     0.948962   
                      3   6084        0.248467        0.947140     0.953644   
                      4   6084        0.210600        0.946724     0.954113   
                      5   6084        0.199033        0.941918     0.944124   
          CmBERT+ptrn 1   6084        0.206024        0.942260     0.957703   
                      2   6084        0.158690        0.946911     0.952084   
                      3   6084        0.174147        0.947524     0.949742   
                      4   6084        0.190652        0.943963     0.951771   
                      5   6084        0.170805        0.943533     0.951928   

                                      eval_f1  eval_accuracy  \
Dataset   Test        run trainsize                            
Reference CmBERT      1   6084       0.964333       0.956089   
                      2   6084       0.969504       0.956244   
                      3   6084       0.969980       0.958050   
                      4   6084       0.967576       0.954334   
                      5   6084       0.968689       0.955212   
          CmBERT+ptrn 1   6084       0.971446       0.957276   
                      2   6084       0.968284       0.956192   
                      3   6084       0.966145       0.956863   
                      4   6084       0.963291       0.952477   
                      5   6084       0.971193       0.956656   
OCR       CmBERT      1   6084       0.944865       0.937099   
                      2   6084       0.947262       0.945868   
                      3   6084       0.950381       0.945451   
                      4   6084       0.950404       0.945973   
                      5   6084       0.943020       0.937099   
          CmBERT+ptrn 1   6084       0.949919       0.944981   
                      2   6084       0.949490       0.945294   
                      3   6084       0.948632       0.945607   
                      4   6084       0.947851       0.942058   
                      5   6084       0.947712       0.947956   

                                                                              eval_PER  \
Dataset   Test        run trainsize                                                      
Reference CmBERT      1   6084       {'precision': 0.965761511216056, 'recall': 0.9...   
                      2   6084       {'precision': 0.9711255156157921, 'recall': 0....   
                      3   6084       {'precision': 0.9793144208037821, 'recall': 0....   
                      4   6084       {'precision': 0.9711084905660371, 'recall': 0....   
                      5   6084       {'precision': 0.9709715639810421, 'recall': 0....   
          CmBERT+ptrn 1   6084       {'precision': 0.9775811209439521, 'recall': 0....   
                      2   6084       {'precision': 0.9741176470588231, 'recall': 0....   
                      3   6084       {'precision': 0.9700352526439481, 'recall'

## Build the averaged table


In [19]:
# Store p/r/f1 as percentages
eval_ = ["eval_f1","eval_precision","eval_recall"]
metrics = metrics_raw.copy()
metrics[eval_] = metrics_raw[eval_].multiply(100., axis=1)
metrics.head()

eval_loss  eval_precision  eval_recall  \
Dataset   Test   run trainsize                                           
Reference CmBERT 1   6084        0.147203       96.455894    96.410737   
                 2   6084        0.167061       96.664598    97.237828   
                 3   6084        0.165029       96.682171    97.315855   
                 4   6084        0.150362       96.652133    96.863296   
                 5   6084        0.181062       96.937021    96.800874   

                                  eval_f1  eval_accuracy  \
Dataset   Test   run trainsize                             
Reference CmBERT 1   6084       96.433310       0.956089   
                 2   6084       96.950366       0.956244   
                 3   6084       96.997978       0.958050   
                 4   6084       96.757599       0.954334   
                 5   6084       96.868900       0.955212   

                                                                         eval_PER  \
Dataset   Test   run trainsize                                                      
Reference CmBERT 1   6084       {'precision': 0.965761511216056, 'recall': 0.9...   
                 2   6084       {'precision': 0.9711255156157921, 'recall': 0....   
                 3   6084       {'precision': 0.9793144208037821, 'recall': 0....   
                 4   6084       {'precision': 0.9711084905660371, 'recall': 0....   
                 5   6084       {'precision': 0.9709715639810421, 'recall': 0....   

                                                                         eval_ACT  \
Dataset   Test   run trainsize                                                      
Reference CmBERT 1   6084       {'precision': 0.9128440366972471, 'recall': 0....   
                 2   6084       {'precision': 0.9204339963833631, 'recall': 0....   
                 3   6084       {'precision': 0.9077887197851381, 'recall': 0....   
                 4   6084       {'precision': 0.9202566452795601, 'recall': 0....   
                 5   6084       {'precision': 0.928305400372439, 'recall': 0.9...   

                                                                         eval_LOC  \
Dataset   Test   run trainsize                                                      
Reference CmBERT 1   6084       {'precision': 0.97265625, 'recall': 0.97537772...   
                 2   6084       {'precision': 0.9693763919821821, 'recall': 0....   
                 3   6084       {'precision': 0.9682981090100111, 'recall': 0....   
                 4   6084       {'precision': 0.9688369504730101, 'recall': 0....   
                 5   6084       {'precision': 0.9710950528071151, 'recall': 0....   

                                                                    eval_CARDINAL  \
Dataset   Test   run trainsize                                                      
Reference CmBERT 1   6084       {'precision': 0.9977155910908051, 'recall': 0....   
                 2   6084       {'precision': 0.9965831435079721, 'recall': 0....   
                 3   6084       {'precision': 0.9982876712328761, 'recall': 0....   
                 4   6084       {'precision': 0.9965753424657531, 'recall': 0....   
                 5   6084       {'precision': 0.9971493728620291, 'recall': 0....   

                                                                          eval_FT  \
Dataset   Test   run trainsize                                                      
Reference CmBERT 1   6084       {'precision': 0.44444444444444403, 'recall': 0...   
                 2   6084       {'precision': 0.583333333333333, 'recall': 0.5...   
                 3   6084       {'precision': 0.5454545454545451, 'recall': 0....   
                 4   6084       {'precision': 0.555555555555555, 'recall': 0.3...   
                 5   6084       {'precision': 0.5454545454545451, 'recall': 0....   

                                                                       eval_TITRE  \
Dataset   Test   run trains

In [11]:
eval_ = []
for elem in metrics_raw.columns:
    if 'eval_p' in elem or 'eval_re' in elem or 'eval_f' in elem or 'eval_ac' in elem:
        eval_.append(elem)

In [24]:
# Average over runs
averaged = metrics.groupby(level=(0,1)).mean()

# Keep just the necessary columns
averaged=averaged[["eval_f1"]]
averaged.rename(columns={"eval_f1":"F1-Score",
                         }, errors="raise", inplace=True)
averaged

F1-Score
Dataset   Test                  
Reference CmBERT       96.801631
          CmBERT+ptrn  96.807198
OCR       CmBERT       94.718631
          CmBERT+ptrn  94.872079

## Create the results table
Title: "Experimental results of the Flat NER models performances on reference dataset"

In [26]:
latex_table = averaged.copy()

caption = "F1 score measured for each approach, pre-trained model and tag format (mean of 5 runs)."
print(latex_table.to_latex(float_format="%.1f", multirow=True, caption=caption))
latex_table

\begin{table}
\centering
\caption{F1 score measured for each approach, pre-trained model and tag format (mean of 5 runs).}
\begin{tabular}{llr}
\toprule
    &             &  F1-Score \\
Dataset & Test &           \\
\midrule
\multirow{2}{*}{Reference} & CmBERT &      96.8 \\
    & CmBERT+ptrn &      96.8 \\
\cline{1-3}
\multirow{2}{*}{OCR} & CmBERT &      94.7 \\
    & CmBERT+ptrn &      94.9 \\
\bottomrule
\end{tabular}
\end{table}



F1-Score
Dataset   Test                  
Reference CmBERT       96.801631
          CmBERT+ptrn  96.807198
OCR       CmBERT       94.718631
          CmBERT+ptrn  94.872079

# 32 - Figures and tables by classe

In [27]:
import pandas as pd
import json
from pandas import json_normalize

def compile_metrics_by_classes(path, classes): 
    rundirs = [f for f in os.listdir(path)]

    df = pd.DataFrame()
    for run_dir in rundirs:
        if 'run' in run_dir:
            run_path = path / run_dir
            nrun = re.search("\d+",run_dir)[0]

            files = [f for f in os.listdir(run_path) if "test_" in f and 'json' in f]
            sizes = [int(re.search("\d+",f)[0]) for f in files]
                
            
            for file, size in zip(files,sizes):
                file_path = run_path / file
                dftmp = pd.read_json(file_path)
                classes_dict = {key: dftmp[key] for key in classes}
                dftmp = pd.DataFrame.from_dict(classes_dict)
                dftmp = dftmp.T
                dftmp['number'] = dftmp['number'].astype(int)
                dftmp["trainsize"] = size 
                dftmp["run"] = nrun
                dftmp["trainsize_p"] = round(100 * size / MAX_TRAINSET_SIZE, 1)
                df = pd.concat([df, dftmp])
                df["classe"] = df.index
                
    return df.groupby(["run","classe"]).first()

In [29]:
classes = ['eval_PER','eval_ACT','eval_LOC','eval_CARDINAL','eval_FT','eval_TITRE']
res = []
keys = []

if os.path.exists(METRICS_DIR_E1 / "21-flat-ner-ref-camembert_ner/run_2"):
    camembert = compile_metrics_by_classes(METRICS_DIR_E1 / "21-flat-ner-ref-camembert_ner",classes)
    res.append(camembert)
    keys.append("CmBERT / ref")

if os.path.exists(METRICS_DIR_E1 / "22-flat-ner-ref-pretrained_camembert_ner/run_2"):
    ptrn_camembert = compile_metrics_by_classes(METRICS_DIR_E1 / "22-flat-ner-ref-pretrained_camembert_ner",classes)
    res.append(ptrn_camembert)
    keys.append("CmBERT+ptrn / ref")
    
if os.path.exists(METRICS_DIR_E2 / "41-flat-ner-pero-ocr-camembert_ner/run_2"):
    camembert_p = compile_metrics_by_classes(METRICS_DIR_E2 / "41-flat-ner-pero-ocr-camembert_ner",classes)
    res.append(camembert_p)
    keys.append("CmBERT / Pero OCR")

if os.path.exists(METRICS_DIR_E2 / "42-flat-ner-pero-ocr-pretrained_camembert_ner/run_2"):
    ptrn_camembert_p = compile_metrics_by_classes(METRICS_DIR_E2 / "42-flat-ner-pero-ocr-pretrained_camembert_ner",classes)
    res.append(ptrn_camembert_p)
    keys.append("CmBERT+ptrn / Pero OCR")

# Create the full table
metrics_raw_classes = pd.concat(res, keys=keys)
metrics_raw_classes.head()

precision    recall        f1  number  \
             run classe                                                 
CmBERT / ref 1   eval_ACT        0.912844  0.909506  0.911172    1094   
                 eval_CARDINAL   0.997716  0.997716  0.997716    1751   
                 eval_FT         0.444444  0.285714  0.347826      14   
                 eval_LOC        0.972656  0.975378  0.974015    1787   
                 eval_PER        0.965762  0.970344  0.968047    1686   

                                trainsize  trainsize_p  
             run classe                                 
CmBERT / ref 1   eval_ACT            6084        100.0  
                 eval_CARDINAL       6084        100.0  
                 eval_FT             6084        100.0  
                 eval_LOC            6084        100.0  
                 eval_PER            6084        100.0

In [30]:
# Store p/r/f1 as percentages
val = ["f1","precision","recall"]
metrics_raw_classes = metrics_raw_classes.copy()
metrics_raw_classes[val] = metrics_raw_classes[val].multiply(100., axis=1)
metrics_raw_classes.head()

precision     recall         f1  number  \
             run classe                                                   
CmBERT / ref 1   eval_ACT       91.284404  90.950640  91.117216    1094   
                 eval_CARDINAL  99.771559  99.771559  99.771559    1751   
                 eval_FT        44.444444  28.571429  34.782609      14   
                 eval_LOC       97.265625  97.537773  97.401509    1787   
                 eval_PER       96.576151  97.034401  96.804734    1686   

                                trainsize  trainsize_p  
             run classe                                 
CmBERT / ref 1   eval_ACT            6084        100.0  
                 eval_CARDINAL       6084        100.0  
                 eval_FT             6084        100.0  
                 eval_LOC            6084        100.0  
                 eval_PER            6084        100.0

In [377]:
# Average over runs
averaged_classes = metrics_raw_classes.groupby(level=0).apply(lambda grp: grp.groupby(by="classe").mean())
averaged_classes.set_index(["number"],
                   append=True,
                   inplace=True)

# Keep just the necessary columns
averaged_classes=averaged_classes[["precision","recall","f1"]]

# Set pretty names
averaged_classes.index.names = ['Model','Entity type',"Count"]
averaged_classes.rename(mapper={'eval_PER':'PER','eval_ACT':'ACT','eval_TITRE':'TITRE','eval_LOC':'LOC','eval_CARDINAL':'CARDINAL','eval_FT':'FT'}, errors="ignore", inplace=True, axis=0)
averaged_classes

precision     recall         f1
Model                  Entity type Count                                  
CmBERT / Pero OCR      ACT         1094.0  89.513516  91.279707  90.385014
                       CARDINAL    1751.0  98.903921  97.841234  98.369267
                       FT          14.0    54.710230  57.142857  54.042375
                       LOC         1787.0  94.136630  95.232233  94.681060
                       PER         1686.0  95.053943  95.516014  95.284307
                       TITRE       75.0    73.588974  66.666667  69.595416
CmBERT / ref           ACT         1094.0  91.792576  91.919561  91.851871
                       CARDINAL    1751.0  99.726222  99.840091  99.783098
                       FT          14.0    53.484848  40.000000  45.621405
                       LOC         1787.0  97.005255  97.515389  97.259609
                       PER         1686.0  97.165630  97.591934  97.378228
                       TITRE       76.0    84.005940  83.684211  83.785011
CmBERT+ptrn / Pero OCR ACT         1094.0  88.119486  91.206581  89.632863
                       CARDINAL    1751.0  98.505350  97.749857  98.125461
                       FT          14.0    58.060606  52.857143  55.125729
                       LOC         1787.0  94.471895  95.601567  95.032955
                       PER         1686.0  95.684971  96.488731  96.084825
                       TITRE       75.0    75.552642  68.800000  71.886942
CmBERT+ptrn / ref      ACT         1094.0  90.906292  92.650823  91.768344
                       CARDINAL    1751.0  99.657749  99.782981  99.720308
                       FT          14.0    55.308913  40.000000  45.915839
                       LOC         1787.0  96.825566  97.616116  97.219107
                       PER         1686.0  97.354064  98.113879  97.732307
                       TITRE       76.0    81.785802  78.947368  80.226774

In [378]:
df2=averaged_classes.reset_index()

In [379]:
def preparetable(model_name):
    cm = df2[df2["Model"] == model_name]
    cm = cm[["Entity type","f1"]]
    cm = cm.reset_index(drop=True)
    cm = cm.T
    cm.columns = cm.iloc[0]
    classes  = list(cm.columns)
    cm = cm[1:]
    cm = list(cm.iloc[0])
    return classes, cm

In [380]:
models = keys.copy()
data = []
for model in models:
    classes, cm = preparetable(model)
    data.append(cm)

final_df = pd.DataFrame(columns=classes, data=data,index=models)
final_df = final_df.reindex(columns=['PER','ACT','LOC','CARDINAL','FT','TITRE'])
final_df

,PER,ACT,LOC,CARDINAL,FT,TITRE
CmBERT / ref,97.378228,91.851871,97.259609,99.783098,45.621405,83.785011
CmBERT+ptrn / ref,97.732307,91.768344,97.219107,99.720308,45.915839,80.226774
CmBERT / Pero OCR,95.284307,90.385014,94.681060,98.369267,54.042375,69.595416
CmBERT+ptrn / Pero OCR,96.084825,89.632863,95.032955,98.125461,55.125729,71.886942


In [361]:
# Filter and transpose to obtain the latex table
latex_table_classes = final_df.copy()

caption = "F1 scores measured for each flat NER class on the fine-tuned models CmBERT and CmBERT+ptrn on reference dataset"
print(latex_table_classes.to_latex(float_format="%.1f", multirow=True, caption=caption))
latex_table_classes

\begin{table}
\centering
\caption{F1 scores measured for each flat NER class on the fine-tuned models CmBERT and CmBERT+ptrn on reference dataset}
\begin{tabular}{lrrrrrr}
\toprule
{} &  PER &  ACT &  LOC &  CARDINAL &   FT &  TITRE \\
\midrule
CmBERT / ref           & 97.4 & 91.9 & 97.3 &      99.8 & 45.6 &   83.8 \\
CmBERT+ptrn / ref      & 97.7 & 91.8 & 97.2 &      99.7 & 45.9 &   80.2 \\
CmBERT / Pero OCR      & 95.3 & 90.4 & 94.7 &      98.4 & 54.0 &   69.6 \\
CmBERT+ptrn / Pero OCR & 96.1 & 89.6 & 95.0 &      98.1 & 55.1 &   71.9 \\
\bottomrule
\end{tabular}
\end{table}



,PER,ACT,LOC,CARDINAL,FT,TITRE
CmBERT / ref,97.378228,91.851871,97.259609,99.783098,45.621405,83.785011
CmBERT+ptrn / ref,97.732307,91.768344,97.219107,99.720308,45.915839,80.226774
CmBERT / Pero OCR,95.284307,90.385014,94.681060,98.369267,54.042375,69.595416
CmBERT+ptrn / Pero OCR,96.084825,89.632863,95.032955,98.125461,55.125729,71.886942
